In [1]:
from pyspark.sql import SparkSession
import os

spark = SparkSession.builder \
    .appName("CRM Data Analysis") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

file_path = r"C:\Users\KIIT\Desktop\synthetic_crm_data.csv"
sql_friendly_path = file_path.replace("\\", "/")

if os.path.isabs(sql_friendly_path) and not sql_friendly_path.startswith("file:///"):
    sql_friendly_path = "file:///" + sql_friendly_path

table_name = "crm_data"

spark.sql(f"DROP TABLE IF EXISTS {table_name}")

try:
    spark.sql(f"""
        CREATE TABLE {table_name}
        USING csv
        OPTIONS (path '{sql_friendly_path}', header 'true', inferSchema 'true')
    """)
    print(f"Table '{table_name}' created successfully using path: '{sql_friendly_path}'")
except Exception as e:
    print(f"Error creating table '{table_name}': {e}")
    spark.stop()
    exit()
    
print(f"Tables in database: {spark.catalog.listTables()}")

try:
    df = spark.table(table_name)
    print(f"Successfully loaded data from table '{table_name}' into DataFrame.")
except Exception as e:
    print(f"Error loading data from table '{table_name}': {e}")
    spark.stop()
    exit()

schema_df = df.schema

schema_details = []
for field in schema_df.fields:
    col_name = field.name
    col_type_str = str(field.dataType)
    
    try:
        sample_values_rows = spark.sql(f"SELECT DISTINCT `{col_name}` FROM {table_name} LIMIT 5").collect()
        sample_values = [row[0] for row in sample_values_rows]
        sample_str = ", ".join([str(val) for val in sample_values if val is not None])
        schema_details.append(f"Column: `{col_name}` (Type: {col_type_str}) - Sample distinct values: [{sample_str}]")
    except Exception as e:
        schema_details.append(f"Column: `{col_name}` (Type: {col_type_str}) - Error fetching sample values: {e}")

try:
    head_df_pandas = df.limit(5).toPandas()
    head_str = head_df_pandas.to_string(index=False)
    print(head_str)
except Exception as e:
    print(f"Error generating DataFrame head: {e}")

schema_details

Table 'crm_data' created successfully using path: 'file:///C:/Users/KIIT/Desktop/synthetic_crm_data.csv'
Tables in database: [Table(name='crm_data', catalog='spark_catalog', namespace=['default'], description=None, tableType='EXTERNAL', isTemporary=False)]
Successfully loaded data from table 'crm_data' into DataFrame.
           Name                              Email                Phone                Company   Industry   Status LastContacted
     John Lewis stevenblackburn@robbins-turner.com           1653970950            Santana LLC     Retail Prospect    2024-06-04
  William Davis            mullinswesley@yahoo.com     740-058-6169x086         Rosario-Thomas Healthcare Customer    2025-03-04
    Kelly Patel             alicejones@pittman.com   447-611-7401x94969 Young, White and Smith Technology     Lead    2024-11-21
Chris Wilkerson          angelvillanueva@gmail.com         821.900.9018                 Wu PLC     Retail     Lead    2024-12-28
   Sherry Small                    

['Column: `Name` (Type: StringType()) - Sample distinct values: [Jeffrey Hunt, Kelly Patel, Linda Meyer, Pamela Young, Lori Ramirez]',
 'Column: `Email` (Type: StringType()) - Sample distinct values: [kirbyarthur@fields-anderson.net, carrie25@jackson-christian.com, munozanthony@wilson-fuller.info, jjackson@wood.biz, melanie22@carrillo.com]',
 'Column: `Phone` (Type: StringType()) - Sample distinct values: [001-280-768-7988x797, +1-345-140-1522x36173, (055)413-6228x641, 5888393372, +1-529-506-5957x6137]',
 'Column: `Company` (Type: StringType()) - Sample distinct values: [King, Bailey and Berry, Hansen Group, Ali, Lee and Case, Pearson-Espinoza, Hernandez-Heath]',
 'Column: `Industry` (Type: StringType()) - Sample distinct values: [Education, Healthcare, Finance, Technology, Retail]',
 'Column: `Status` (Type: StringType()) - Sample distinct values: [Prospect, Lead, Customer, Churned]',
 'Column: `LastContacted` (Type: DateType()) - Sample distinct values: [2025-02-16, 2024-06-04, 2024-

In [2]:
from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate, ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from openai import OpenAI
from langchain_core.output_parsers import StrOutputParser
import traceback


client = OpenAI(
    base_url='http://localhost:11434/v1',
    api_key='ollama'
)

user_query = [
    "Count of customers by status",
    "Count of customers by industry",
    "List of all churned customers",
    "Customers who haven't been contacted in the last 60 days"
]

system_template = """You are an expert SQL query generator specializing in Spark SQL syntax. 
Generate precise, optimized SQL queries based on the schema provided.
Always follow these rules:
1. Only return the SQL query with no explanations or markdown formatting
2. For date calculations, use date_sub(current_date(), n) for subtracting days
3. Do not use DATE('now'), NOW(), or MySQL/SQLite-style date functions
4. Use simple capital letters for main query aliases
5. Always reference the table as 'crm_data'
6. Use proper date functions compatible with Spark SQL
7. Never modify the schema
8. Be consistent in your query generation approach
"""


human_template = """Based on the schema and sample data below, write a SQL query that answers this question: {user_query}

SCHEMA DETAILS:
{schema_str}

SAMPLE DATA:
{head_str}

SQL QUERY:"""

chat_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template(human_template)
])


llm1 = ChatOpenAI(
    model="gemma3",
    openai_api_base="http://localhost:11434/v1",
    openai_api_key="ollama"
)

parser = StrOutputParser()
result = chat_prompt | llm1 | parser


def generate_query(user_queries):
    sql_queries = []
    for query in user_queries:
        sql = result.invoke({
            "schema_str": "\n".join(schema_details),
            "head_str": head_str,
            "user_query": query
        })
        sql = sql.replace("```sql", "").replace("```", "").strip()
        sql_queries.append(sql)
    return sql_queries


check_query_prompt = PromptTemplate.from_template("""
Check if the following SQL query is valid for the given schema and Spark SQL. 
If any error is found, fix the query as per the schema and Spark SQL requirements.
Return only the corrected SQL query, nothing else.

Schema:
{schema_str}

SQL Query:
{sql_query}
""")

checker_chain = check_query_prompt | llm1 | parser

def check(sql_queries_tuple):
    checked_queries = []
    for query in sql_queries_tuple:
        checked_sql = checker_chain.invoke({
            "schema_str": "\n".join(schema_details),
            "sql_query": query
        })
        checked_queries.append(checked_sql)
    return tuple(checked_queries)

def execute_query_with_error_capture(sql_query, spark):
    try:
        result_df = spark.sql(sql_query)
        return {"success": True, "result": result_df}
    except Exception as e:
        tb_str = traceback.format_exc()
        return {"success": False, "error_message": str(e), "traceback": tb_str}

error_fix_prompt = PromptTemplate.from_template("""
The following SQL query was executed in Spark SQL and resulted in an error.
Please fix the query so it works, considering the schema and error message below.
Return only the corrected SQL query, in Spark SQL syntax.

Schema:
{schema_str}

SQL Query:
{sql_query}

Error Message:
{error_message}
""")

def auto_fix_and_execute(sql_query, spark, schema_details, max_attempts=5):
    attempts = 0
    current_sql = sql_query
    while attempts < max_attempts:
        result = execute_query_with_error_capture(current_sql, spark)
        if result["success"]:
            return result["result"]
        else:
            corrected_chain = error_fix_prompt | llm1 | parser
            current_sql = corrected_chain.invoke({
                "schema_str": "\n".join(schema_details),
                "sql_query": current_sql,
                "error_message": result["error_message"]
            })
        attempts += 1
    raise Exception("Failed to generate a correct SQL query after multiple attempts.")

def execute():
    results = generate_query(user_query)
    final = check(results)
    for idx, query in enumerate(final, start=1):
        print(f"\nQuery {idx}:")
        print(f"SQL: {query}")
        try:
            result_df = auto_fix_and_execute(query, spark, schema_details)
            print(result_df)
        except Exception as e:
            print(f"Failed to execute corrected query after several attempts: {e}")

execute()
spark.stop()


Query 1:
SQL: SELECT
    Status,
    COUNT(*) AS CustomerCount
FROM
    crm_data
GROUP BY
    Status;

DataFrame[Status: string, CustomerCount: bigint]

Query 2:
SQL: SELECT Industry, count(*) FROM crm_data GROUP BY Industry

DataFrame[Industry: string, count(1): bigint]

Query 3:
SQL: SELECT
  Name
FROM crm_data
WHERE
  Status = 'Churned';

DataFrame[Name: string]

Query 4:
SQL: SELECT Name, Email, Phone, Company, Industry, Status, LastContacted
FROM crm_data
WHERE LastContacted < date_sub(current_date(), 60);
DataFrame[Name: string, Email: string, Phone: string, Company: string, Industry: string, Status: string, LastContacted: date]


In [5]:
import pandas as pd
import panel as pn
import plotly.express as px
import plotly.graph_objects as go
import json
import holoviews as hv
from panel.template import MaterialTemplate
import param


viz_system_template = """You are a data visualization expert. Recommend the best visualization 
for the given SQL and results as a JSON with these fields: 
{
  "visualization_type": "bar|line|pie|scatter|table",
  "title": "Title",
  "x_axis": "x column",
  "y_axis": "y column",
  "description": "Brief explanation"
}"""

viz_human_template = """Recommend visualization for:
SQL: {sql_query}
RESULTS: {query_results}
JSON ONLY:"""

viz_chat_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(viz_system_template),
    HumanMessagePromptTemplate.from_template(viz_human_template)
])

viz_recommender = viz_chat_prompt | client | parser

query_input = pn.widgets.TextAreaInput(
    name="Query",
    placeholder="Enter your query about the CRM data",
    value="Count of customers by status"
)

result_display = pn.pane.Markdown("Results will appear here")
viz_display = pn.pane.Plotly(height=400)


def get_visualization_recommendation(sql_query, query_results_df, user_query):
    # Convert results to string representation for the LLM
    results_str = query_results_df.limit(10).toPandas().to_string(index=False)
    
    viz_recommendation = viz_recommender.invoke({
        "sql_query": sql_query,
        "query_results": results_str,
        "user_query": user_query
    })
    
    try:
        # Parse JSON recommendation
        viz_config = json.loads(viz_recommendation)
        return viz_config
    except json.JSONDecodeError:
        # If parsing fails, use a simpler approach to extract JSON
        start_idx = viz_recommendation.find('{')
        end_idx = viz_recommendation.rfind('}') + 1
        if start_idx >= 0 and end_idx > start_idx:
            json_str = viz_recommendation[start_idx:end_idx]
            try:
                viz_config = json.loads(json_str)
                return viz_config
            except:
                pass
        
        # Fallback to default configuration
        return {
            "visualization_type": "table",
            "title": "Query Results",
            "description": "Unable to determine optimal visualization. Showing table view."
        }

# Function to create visualization
def create_visualization(viz_config, results_df):
    # Convert Spark DataFrame to pandas
    pdf = results_df.toPandas()
    
    viz_type = viz_config.get("visualization_type", "table").lower()
    title = viz_config.get("title", "Query Results")
    
    if viz_type == "table":
        return pn.pane.DataFrame(pdf, sizing_mode='stretch_width')
    
    elif viz_type == "bar":
        x = viz_config.get("x_axis")
        y = viz_config.get("y_axis")
        color_by = viz_config.get("color_by")
        
        if color_by:
            fig = px.bar(pdf, x=x, y=y, color=color_by, title=title)
        else:
            fig = px.bar(pdf, x=x, y=y, title=title)
        
        return pn.pane.Plotly(fig, sizing_mode='stretch_width')
    
    elif viz_type == "line":
        x = viz_config.get("x_axis")
        y = viz_config.get("y_axis")
        color_by = viz_config.get("color_by")
        
        if color_by:
            fig = px.line(pdf, x=x, y=y, color=color_by, title=title)
        else:
            fig = px.line(pdf, x=x, y=y, title=title)
        
        return pn.pane.Plotly(fig, sizing_mode='stretch_width')
    
    elif viz_type == "pie":
        values = viz_config.get("y_axis")
        names = viz_config.get("x_axis")
        
        fig = px.pie(pdf, values=values, names=names, title=title)
        return pn.pane.Plotly(fig, sizing_mode='stretch_width')
    
    elif viz_type == "scatter":
        x = viz_config.get("x_axis")
        y = viz_config.get("y_axis")
        color_by = viz_config.get("color_by")
        
        if color_by:
            fig = px.scatter(pdf, x=x, y=y, color=color_by, title=title)
        else:
            fig = px.scatter(pdf, x=x, y=y, title=title)
        
        return pn.pane.Plotly(fig, sizing_mode='stretch_width')
    
    elif viz_type == "heatmap":
        x = viz_config.get("x_axis")
        y = viz_config.get("y_axis")
        z = viz_config.get("z_axis", viz_config.get("y_axis"))
        
        # Pivot data for heatmap if needed
        if len(pdf[x].unique()) > 1 and len(pdf[y].unique()) > 1:
            pivot_df = pdf.pivot_table(values=z, index=y, columns=x, aggfunc='sum')
            fig = px.imshow(pivot_df, title=title)
        else:
            fig = px.density_heatmap(pdf, x=x, y=y, title=title)
        
        return pn.pane.Plotly(fig, sizing_mode='stretch_width')
    
    else:
        # Default to table if visualization type not recognized
        return pn.pane.DataFrame(pdf, sizing_mode='stretch_width')

# Dashboard class
class CRMDashboard(param.Parameterized):
    query_text = param.String(default="", doc="Enter your query about the CRM data")
    run_query = param.Action(lambda x: x.param.trigger('run_query'), doc="Generate SQL and Visualization")
    
    def __init__(self, **params):
        super().__init__(**params)
        self.file_path = r"C:\Users\KIIT\Desktop\synthetic_crm_data.csv"
        self.df = file_path
        self.schema_details, self.head_str = schema_details, head_str
        
        # Predefined queries
        self.predefined_queries = [
            "Count of customers by status",
            "Count of customers by industry",
            "List of all churned customers",
            "Customers who haven't been contacted in the last 60 days",
            "Average time since last contact by status",
            "Industry distribution of prospects",
            "Monthly trend of customer acquisition"
        ]
        
        self.query_result = pn.pane.Markdown("Enter a query and click 'Run' to see results")
        self.visualization = pn.pane.Markdown("Visualization will appear here")
        self.sql_display = pn.pane.Markdown("SQL query will appear here")
        
    def _run_query(self, event):
        if not self.query_text:
            self.query_result.object = "Please enter a query first"
            return
        
        # Generate and fix SQL query
        sql_query = generate_sql_query(self.query_text, self.schema_details, self.head_str)
        sql_query = check_query(sql_query, self.schema_details)
        
        try:
            # Execute query
            self.sql_display.object = f"```sql\n{sql_query}\n```"
            results_df = auto_fix_and_execute(sql_query, spark, self.schema_details)
            
            # Get visualization recommendation
            viz_config = get_visualization_recommendation(sql_query, results_df, self.query_text)
            
            # Create visualization
            visualization = create_visualization(viz_config, results_df)
            self.visualization.object = visualization
            
            # Show summary of results
            count = results_df.count()
            summary = f"Query returned {count} results. "
            summary += viz_config.get("description", "")
            self.query_result.object = summary
            
        except Exception as e:
            self.query_result.object = f"Error executing query: {str(e)}"
    
    @param.depends('run_query', watch=True)
    def _trigger_run(self):
        self._run_query(None)
    
    def panel(self):
        # Query input section
        query_input = pn.widgets.TextAreaInput(
            name="Query",
            placeholder="Enter your query about the CRM data",
            value=self.query_text,
            sizing_mode='stretch_width'
        )
        query_input.link(self, bidirectional=True)
        
        # Predefined queries dropdown
        predefined = pn.widgets.Select(
            name='Predefined Queries',
            options=self.predefined_queries,
            sizing_mode='stretch_width'
        )
        
        def update_query(event):
            self.query_text = event.new
        
        predefined.param.watch(update_query, 'value')
        
        # Run button
        run_button = pn.widgets.Button(
            name='Run Query',
            button_type='primary',
            sizing_mode='stretch_width'
        )
        
        def trigger_run(event):
            self._run_query(event)
        
        run_button.on_click(trigger_run)
        
        # Create layout
        inputs = pn.Column(
            "## CRM Data Analysis",
            predefined,
            query_input,
            run_button,
            sizing_mode='stretch_width'
        )
        
        sql_card = pn.Card(
            self.sql_display,
            title="Generated SQL",
            sizing_mode='stretch_width'
        )
        
        viz_card = pn.Card(
            self.visualization,
            title="Visualization",
            sizing_mode='stretch_width'
        )
        
        result_card = pn.Card(
            self.query_result,
            title="Result Summary",
            sizing_mode='stretch_width'
        )
        
        main_content = pn.Column(
            result_card,
            pn.Row(
                sql_card,
                sizing_mode='stretch_width'
            ),
            viz_card,
            sizing_mode='stretch_width'
        )
        
        sidebar = pn.Column(
            inputs,
            "### Schema Information",
            pn.pane.Markdown("\n".join(self.schema_details[:3]) + "\n\n*[...more fields available]*"),
            sizing_mode='stretch_width'
        )
        
        template = MaterialTemplate(
            title="Dynamic CRM Dashboard",
            sidebar=sidebar,
            main=main_content
        )
        
        return template

# Function to launch the dashboard
def launch_dashboard():
    dashboard = CRMDashboard()
    return dashboard.panel().servable()

if __name__ == "__main__":
    launch_dashboard()


ValueError: Declare parameters to link or a set of callbacks, neither was defined.